# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::359346771395:role/service-role/AmazonSageMaker-ExecutionRole-20240516T125452


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://udacity-selfdriving-240516-1358/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [ ]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
Sending build context to Docker daemon  725.6MB
Step 1/14 : FROM tensorflow/tensorflow:2.13.0-gpu
 ---> 6bdca089cc38
Step 2/14 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in 6eedfe1e1c4f
Removing intermediate container 6eedfe1e1c4f
 ---> 70854d3886b9
Step 3/14 : RUN apt-get update && apt-get install -y     git     gpg-agent     python3-cairocffi     protobuf-compiler     python3-pil     python3-lxml     python3-tk     libgl1-mesa-dev     wget
 ---> Running in 125174d869df
Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease 

Get:16 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-6 amd64 2:1.6.9-2ubuntu1.6 [577 kB]
Get:17 http://archive.ubuntu.com/ubuntu focal/main amd64 libxext6 amd64 2:1.3.4-0ubuntu1 [29.1 kB]
Get:18 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmuu1 amd64 2:1.1.3-0ubuntu1 [9728 B]
Get:19 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openssh-client amd64 1:8.2p1-4ubuntu0.11 [670 kB]
Get:20 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 wget amd64 1.20.3-1ubuntu2 [348 kB]
Get:21 http://archive.ubuntu.com/ubuntu focal/main amd64 xauth amd64 1:1.1-0ubuntu1 [25.0 kB]
Get:22 http://archive.ubuntu.com/ubuntu focal/main amd64 libtcl8.6 amd64 8.6.10+dfsg-1 [902 kB]
Get:23 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libfreetype6 amd64 2.10.1-2ubuntu0.3 [341 kB]
Get:24 http://archive.ubuntu.com/ubuntu focal/main amd64 fonts-dejavu-core all 2.37-1 [1041 kB]
Get:25 http://archive.ubuntu.com/ubuntu focal/main amd64 fontconfig-config all 2.1

Get:99 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libprotobuf-lite17 amd64 3.6.1.3-2ubuntu5.2 [132 kB]
Get:100 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libprotobuf17 amd64 3.6.1.3-2ubuntu5.2 [798 kB]
Get:101 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libprotoc17 amd64 3.6.1.3-2ubuntu5.2 [646 kB]
Get:102 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebpdemux2 amd64 0.6.1-2ubuntu0.20.04.3 [9560 B]
Get:103 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebpmux3 amd64 0.6.1-2ubuntu0.20.04.3 [19.5 kB]
Get:104 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-randr0 amd64 1.14-2 [16.3 kB]
Get:105 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxslt1.1 amd64 1.1.34-4ubuntu0.20.04.1 [151 kB]
Get:106 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 mesa-vulkan-drivers amd64 21.2.6-0ubuntu0.1~20.04.2 [5788 kB]
Get:107 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-soupsieve all 1.

Selecting previously unselected package tk8.6-blt2.5.
Preparing to unpack .../031-tk8.6-blt2.5_2.5.3+dfsg-4_amd64.deb ...
Unpacking tk8.6-blt2.5 (2.5.3+dfsg-4) ...
Selecting previously unselected package blt.
Preparing to unpack .../032-blt_2.5.3+dfsg-4_amd64.deb ...
Unpacking blt (2.5.3+dfsg-4) ...
Selecting previously unselected package libcurl3-gnutls:amd64.
Preparing to unpack .../033-libcurl3-gnutls_7.68.0-1ubuntu2.22_amd64.deb ...
Unpacking libcurl3-gnutls:amd64 (7.68.0-1ubuntu2.22) ...
Selecting previously unselected package liberror-perl.
Preparing to unpack .../034-liberror-perl_0.17029-1_all.deb ...
Unpacking liberror-perl (0.17029-1) ...
Selecting previously unselected package git-man.
Preparing to unpack .../035-git-man_1%3a2.25.1-1ubuntu3.11_all.deb ...
Unpacking git-man (1:2.25.1-1ubuntu3.11) ...
Selecting previously unselected package git.
Preparing to unpack .../036-git_1%3a2.25.1-1ubuntu3.11_amd64.deb ...
Unpacking git (1:2.25.1-1ubuntu3.11) ...
Selecting previously un

Selecting previously unselected package libpthread-stubs0-dev:amd64.
Preparing to unpack .../077-libpthread-stubs0-dev_0.4-1_amd64.deb ...
Unpacking libpthread-stubs0-dev:amd64 (0.4-1) ...
Selecting previously unselected package libxcb1-dev:amd64.
Preparing to unpack .../078-libxcb1-dev_1.14-2_amd64.deb ...
Unpacking libxcb1-dev:amd64 (1.14-2) ...
Selecting previously unselected package libx11-dev:amd64.
Preparing to unpack .../079-libx11-dev_2%3a1.6.9-2ubuntu1.6_amd64.deb ...
Unpacking libx11-dev:amd64 (2:1.6.9-2ubuntu1.6) ...
Selecting previously unselected package libglx-dev:amd64.
Preparing to unpack .../080-libglx-dev_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
Unpacking libglx-dev:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Selecting previously unselected package libgl-dev:amd64.
Preparing to unpack .../081-libgl-dev_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
Unpacking libgl-dev:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Selecting previously unselected package libegl-dev:amd64.
Preparing to unpack .../082

...

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.32.0
    Uninstalling tensorflow-io-gcs-filesystem-0.32.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.32.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2019.11.28
    Uninstalling certifi-2019.11.28:
      Successfully uninstalled certifi-2019.11.28
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
Removing intermediate container 9ecfbba267ad
 ---> e31f56901b27
Step 12/14 : ENV TF_CPP_MIN_LOG_LEVEL 3
 ---

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 32.6 MB/s eta 0:00:00
  Created wheel for sagemaker-training: filename=sagemaker_training-4.7.4-cp38-cp38-linux_x86_64.whl size=93695 sha256=3cd742dc8103e59cdf910efab0a4b99116853c84e486646fd260e13a234365d2
  Stored in directory: /root/.cache/pip/wheels/ba/ef/65/f75eb4b9f6df895e991783a6ad8bf1827759539e15227a2ee5
  Created wheel for inotify_simple: filename=inotify_simple-1.2.1-py3-none-any.whl size=8201 sha256=c379e4b6954731fb56ddf9bdea2442dab8dfdd8afe828597e5bd23f611413438
  Stored in directory: /root/.cache/pip/wheels/8b/2d/c2/46bac8503a2469925f6f463615984b3d1fe472e729363a28d3
Successfully built sagemaker-training inotify_simple
Removing intermediate container 23fc63ef13b0
 ---> 673473d7b9f7
Step 14/14 : RUN pip install "opencv-python-headless<4.3"
 ---> Running in e76ffd680986
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 73.8 MB/s eta 0

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [14]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

359346771395.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20240527120237


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [15]:
%%bash
pwd
rm -r /tmp/checkpoint
rm -r source_dir/checkpoint
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

#wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

#ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
#wget -O /tmp/ssdmobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/ssdmobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint
# result out of memory with pipeline_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config
# will also test with: SSD_MobileNet_from_udacity_mentor.config

# Faster R-CNN ResNet152 V1 640x640
#wget -O /tmp/fasterrcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

#SSD ResNet50 V1 FPN 640x640 (RetinaNet50)
wget -O /tmp/ssdresnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssdresnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

# Faster R-CNN ResNet50 V1 640x640
#wget -O /tmp/fasterrcnn50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint


/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training


--2024-05-27 12:30:02--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.115.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘/tmp/fasterrcnn50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 11.0M 18s
    50K .......... .......... .......... .......... ..........  0% 21.9M 14s
   100K .......... .......... .......... .......... ..........  0% 23.0M 12s
   150K .......... .......... .......... .......... ..........  0% 17.4M 12s
   200K .......... .......... .......... .......... ..........  0% 22.8M 11s
   250K .......... .......... .......... .......... ..........  0% 22.9M 11s
   300K .......... .......... .......... ......

  5100K .......... .......... .......... .......... ..........  2% 36.9M 6s
  5150K .......... .......... .......... .......... ..........  2%  140M 6s
  5200K .......... .......... .......... .......... ..........  2% 37.1M 6s
  5250K .......... .......... .......... .......... ..........  2% 42.0M 6s
  5300K .......... .......... .......... .......... ..........  2% 36.8M 6s
  5350K .......... .......... .......... .......... ..........  2% 36.5M 6s
  5400K .......... .......... .......... .......... ..........  2%  151M 6s
  5450K .......... .......... .......... .......... ..........  2% 35.5M 6s
  5500K .......... .......... .......... .......... ..........  2% 39.8M 6s
  5550K .......... .......... .......... .......... ..........  2% 40.4M 6s
  5600K .......... .......... .......... .......... ..........  2% 39.1M 6s
  5650K .......... .......... .......... .......... ..........  2%  122M 6s
  5700K .......... .......... .......... .......... ..........  2% 38.4M 6s
  5750K ....

 10500K .......... .......... .......... .......... ..........  5%  186M 5s
 10550K .......... .......... .......... .......... ..........  5% 38.3M 5s
 10600K .......... .......... .......... .......... ..........  5% 45.3M 5s
 10650K .......... .......... .......... .......... ..........  5% 44.9M 5s
 10700K .......... .......... .......... .......... ..........  5%  422M 5s
 10750K .......... .......... .......... .......... ..........  5% 44.8M 5s
 10800K .......... .......... .......... .......... ..........  5% 45.4M 5s
 10850K .......... .......... .......... .......... ..........  5% 49.7M 5s
 10900K .......... .......... .......... .......... ..........  5% 51.2M 5s
 10950K .......... .......... .......... .......... ..........  5%  267M 5s
 11000K .......... .......... .......... .......... ..........  5% 53.5M 5s
 11050K .......... .......... .......... .......... ..........  5% 54.4M 5s
 11100K .......... .......... .......... .......... ..........  5% 56.4M 5s
 11150K ....

 15900K .......... .......... .......... .......... ..........  7%  149M 4s
 15950K .......... .......... .......... .......... ..........  7%  233M 4s
 16000K .......... .......... .......... .......... ..........  7%  132M 4s
 16050K .......... .......... .......... .......... ..........  7%  135M 4s
 16100K .......... .......... .......... .......... ..........  7%  134M 4s
 16150K .......... .......... .......... .......... ..........  7%  225M 4s
 16200K .......... .......... .......... .......... ..........  7%  132M 4s
 16250K .......... .......... .......... .......... ..........  7%  125M 4s
 16300K .......... .......... .......... .......... ..........  7%  139M 4s
 16350K .......... .......... .......... .......... ..........  7% 1.48M 4s
 16400K .......... .......... .......... .......... ..........  7% 20.9M 4s
 16450K .......... .......... .......... .......... ..........  7% 44.1M 4s
 16500K .......... .......... .......... .......... ..........  7%  223M 4s
 16550K ....

...

129300K .......... .......... .......... .......... .......... 62%  215M 1s
129350K .......... .......... .......... .......... .......... 62% 80.5M 1s
129400K .......... .......... .......... .......... .......... 62%  225M 1s
129450K .......... .......... .......... .......... .......... 62%  157M 1s
129500K .......... .......... .......... .......... .......... 62% 88.2M 1s
129550K .......... .......... .......... .......... .......... 62%  137M 1s
129600K .......... .......... .......... .......... .......... 62%  153M 1s
129650K .......... .......... .......... .......... .......... 62%  226M 1s
129700K .......... .......... .......... .......... .......... 62% 76.2M 1s
129750K .......... .......... .......... .......... .......... 62%  236M 1s
129800K .......... .......... .......... .......... .......... 62%  144M 1s
129850K .......... .......... .......... .......... .......... 62%  155M 1s
129900K .......... .......... .......... .......... .......... 62%  288M 1s
129950K ....

134700K .......... .......... .......... .......... .......... 65%  225M 1s
134750K .......... .......... .......... .......... .......... 65%  199M 1s
134800K .......... .......... .......... .......... .......... 65%  326M 1s
134850K .......... .......... .......... .......... .......... 65%  175M 1s
134900K .......... .......... .......... .......... .......... 65% 73.2M 1s
134950K .......... .......... .......... .......... .......... 65% 97.2M 1s
135000K .......... .......... .......... .......... .......... 65%  193M 1s
135050K .......... .......... .......... .......... .......... 65%  156M 1s
135100K .......... .......... .......... .......... .......... 65%  335M 1s
135150K .......... .......... .......... .......... .......... 65%  155M 1s
135200K .......... .......... .......... .......... .......... 65%  338M 1s
135250K .......... .......... .......... .......... .......... 65%  337M 1s
135300K .......... .......... .......... .......... .......... 65%  217M 1s
135350K ....

140100K .......... .......... .......... .......... .......... 67%  135M 1s
140150K .......... .......... .......... .......... .......... 67%  161M 1s
140200K .......... .......... .......... .......... .......... 67%  188M 1s
140250K .......... .......... .......... .......... .......... 67%  223M 1s
140300K .......... .......... .......... .......... .......... 67%  152M 1s
140350K .......... .......... .......... .......... .......... 67%  294M 1s
140400K .......... .......... .......... .......... .......... 67%  150M 1s
140450K .......... .......... .......... .......... .......... 67%  188M 1s
140500K .......... .......... .......... .......... .......... 67%  236M 1s
140550K .......... .......... .......... .......... .......... 67%  172M 1s
140600K .......... .......... .......... .......... .......... 67%  238M 1s
140650K .......... .......... .......... .......... .......... 67%  142M 1s
140700K .......... .......... .......... .......... .......... 67%  132M 1s
140750K ....

145500K .......... .......... .......... .......... .......... 70%  229M 1s
145550K .......... .......... .......... .......... .......... 70%  163M 1s
145600K .......... .......... .......... .......... .......... 70%  104M 1s
145650K .......... .......... .......... .......... .......... 70%  192M 1s
145700K .......... .......... .......... .......... .......... 70%  268M 1s
145750K .......... .......... .......... .......... .......... 70%  144M 1s
145800K .......... .......... .......... .......... .......... 70%  226M 1s
145850K .......... .......... .......... .......... .......... 70%  133M 1s
145900K .......... .......... .......... .......... .......... 70%  155M 1s
145950K .......... .......... .......... .......... .......... 70%  164M 1s
146000K .......... .......... .......... .......... .......... 70%  218M 1s
146050K .......... .......... .......... .......... .......... 70%  110M 1s
146100K .......... .......... .......... .......... .......... 70%  195M 1s
146150K ....

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge', # ml.g5.xlarge
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2024-05-27-12-30-07-653


2024-05-27 12:30:13 Starting - Starting the training job......
2024-05-27 12:30:50 Starting - Preparing the instances for training...
2024-05-27 12:31:37 Downloading - Downloading the training image......

In [ ]:
#ls /tmp/checkpoint
#mkdir source_dir/checkpoint

import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

# Extract the directory name
directory_name = os.path.basename(current_directory)

# Print the directory name
print(directory_name)

files = [f for f in os.listdir('/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir/checkpoint')]
for f in files:
    print(f)
    # do something

print("\n/opt")
files = [f for f in os.listdir('/opt/')]
for f in files:
    print(f)
    # do something

    
# /home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your write-up goes here.